**In order to get the results, you need to run everything up to Model Training and Results Section, and then run all the code blocks with the face2sketch comment for the face2sketch training and results, and after getting the results, execute the sketch2face blocks for the sketch2face results.**

We start by importing the used libraries

In [1]:
#https://www.kaggle.com/theblackmamba31/photo-to-sketch-using-autoencoder/notebook
import numpy as np
import tensorflow as tf
import keras 
from keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Input
from keras.preprocessing.image import img_to_array
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm 
import os
import re

In [2]:
#https://drive.google.com/drive/folders/1C2lQ61MssBmVjRxrzYy0vqDOMWWxSEex?usp=sharing Get the files into your drive and open from there
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
def sorted_alphanumeric(data):  
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)',key)]
    return sorted(data,key = alphanum_key)


# defining the size of image 
SIZE = 256

image_path = "/content/gdrive/MyDrive/Data_for_CW1/photos"
img_array = []

sketch_path = '/content/gdrive/MyDrive/Data_for_CW1/sketches'
sketch_array = []


image_file = sorted_alphanumeric(os.listdir(image_path))
sketch_file = sorted_alphanumeric(os.listdir(sketch_path))

Load data.

We convert the images to arrays ansd then we store them in a list

Data augumentation is also performed due to the small amount of provided samples

In [4]:
for i in tqdm(image_file):
    image = cv2.imread(image_path + '/' + i,1)
    
    # as opencv load image in bgr format converting it to rgb
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # resizing images 
    image = cv2.resize(image, (SIZE, SIZE))
    
    # normalizing image 
    image = image.astype('float32') / 255.0
    
    #appending normal normal image    
    img_array.append(img_to_array(image))
    # Image Augmentation
    
    # horizontal flip 
    img1 = cv2.flip(image,1)
    img_array.append(img_to_array(img1))
     #vertical flip 
    img2 = cv2.flip(image,-1)
    img_array.append(img_to_array(img2))
     #vertical flip 
    img3 = cv2.flip(image,-1)
    # horizontal flip
    img3 = cv2.flip(img3,1)
    img_array.append(img_to_array(img3))
    # rotate clockwise 
    img4 = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
    img_array.append(img_to_array(img4))
    # flip rotated image 
    img5 = cv2.flip(img4,1)
    img_array.append(img_to_array(img5))
     # rotate anti clockwise 
    img6 = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
    img_array.append(img_to_array(img6))
    # flip rotated image 
    img7 = cv2.flip(img6,1)
    img_array.append(img_to_array(img7))
  
    
for i in tqdm(sketch_file):
    image = cv2.imread(sketch_path + '/' + i,1)
    
    # as opencv load image in bgr format converting it to rgb
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # resizing images 
    image = cv2.resize(image, (SIZE, SIZE))
    
    # normalizing image 
    image = image.astype('float32') / 255.0
    # appending normal sketch image
    sketch_array.append(img_to_array(image))
    
    #Image Augmentation
    # horizontal flip 
    img1 = cv2.flip(image,1)
    sketch_array.append(img_to_array(img1))
     #vertical flip 
    img2 = cv2.flip(image,-1)
    sketch_array.append(img_to_array(img2))
     #vertical flip 
    img3 = cv2.flip(image,-1)
    # horizontal flip
    img3 = cv2.flip(img3,1)
    sketch_array.append(img_to_array(img3))
    # rotate clockwise 
    img4 = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
    sketch_array.append(img_to_array(img4))
    # flip rotated image 
    img5 = cv2.flip(img4,1)
    sketch_array.append(img_to_array(img5))
     # rotate anti clockwise 
    img6 = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
    sketch_array.append(img_to_array(img6))
    # flip rotated image 
    img7 = cv2.flip(img6,1)
    sketch_array.append(img_to_array(img7))

100%|██████████| 188/188 [01:06<00:00,  2.83it/s]


In [5]:
print("Total number of sketch images:",len(sketch_array))
print("Total number of images:",len(img_array))
print(len(img_array[1]))

Total number of sketch images: 1504
Total number of images: 1504
256


Slicing and reshaping

Out of 1504 images I have sliced them to two part. train images consist 1400 images while test images contains 104 images. After slicing image array, I reshaped them so that images can be fed directly into our encoder network

In [6]:
train_sketch_image = sketch_array[:1400]
train_image = img_array[:1400]
test_sketch_image = sketch_array[1400:]
test_image = img_array[1400:]
# reshaping
train_sketch_image = np.reshape(train_sketch_image,(len(train_sketch_image),SIZE,SIZE,3))
train_image = np.reshape(train_image, (len(train_image),SIZE,SIZE,3))
print('Train color image shape:',train_image.shape)
test_sketch_image = np.reshape(test_sketch_image,(len(test_sketch_image),SIZE,SIZE,3))
test_image = np.reshape(test_image, (len(test_image),SIZE,SIZE,3))
print('Test color image shape',test_image.shape)

Train color image shape: (1400, 256, 256, 3)
Test color image shape (104, 256, 256, 3)


In [15]:
#Load the model if you dont want to run the process again
from tensorflow.keras.models import load_model
model = load_model('/content/gdrive/MyDrive/Data_for_CW1/MyModels/Conv_AE/sketch2faceConvAE.h5')

**Downsample layer**

In [ ]:
def downsample(filters, size, apply_batch_normalization = True):
    downsample = tf.keras.models.Sequential()
    downsample.add(keras.layers.Conv2D(filters = filters, kernel_size = size, strides = 2, use_bias = False, kernel_initializer = 'he_normal'))
    if apply_batch_normalization:
        downsample.add(keras.layers.BatchNormalization())
    downsample.add(keras.layers.LeakyReLU())
    return downsample

**Upsample Layer**

In [ ]:
def upsample(filters, size, apply_dropout = False):
    upsample = tf.keras.models.Sequential()
    upsample.add(keras.layers.Conv2DTranspose(filters = filters, kernel_size = size, strides = 2, use_bias = False, kernel_initializer = 'he_normal'))
    if apply_dropout:
        upsample.add(tf.keras.layers.Dropout(0.1))
    upsample.add(tf.keras.layers.LeakyReLU()) 
    return upsample

**Model**

In [ ]:
def model():
    encoder_input = keras.Input(shape = (SIZE, SIZE, 3))
    x = downsample(16, 4, False)(encoder_input)
    x = downsample(32,4)(x)
    x = downsample(64,4,False)(x)
    x = downsample(128,4)(x)
    x = downsample(256,4)(x)
   
    encoder_output = downsample(512,4)(x)
    
    decoder_input = upsample(512,4,True)(encoder_output)
    x = upsample(256,4,False)(decoder_input)
    x = upsample(128,4, True)(x)
    x = upsample(64,4)(x)
    x = upsample(32,4)(x)
    x = upsample(16,4)(x)
    x = tf.keras.layers.Conv2DTranspose(8,(2,2),strides = (1,1), padding = 'valid')(x)
    decoder_output = tf.keras.layers.Conv2DTranspose(3,(2,2),strides = (1,1), padding = 'valid')(x)
    
  
    return tf.keras.Model(encoder_input, decoder_output)

To get summary of model


In [ ]:
model = model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 127, 127, 16)      768       
                                                                 
 sequential_1 (Sequential)   (None, 62, 62, 32)        8320      
                                                                 
 sequential_2 (Sequential)   (None, 30, 30, 64)        32768     
                                                                 
 sequential_3 (Sequential)   (None, 14, 14, 128)       131584    
                                                                 
 sequential_4 (Sequential)   (None, 6, 6, 256)         525312    
                                                                 
 sequential_5 (Sequential)   (None, 2, 2, 512)         209920

Compiling and Fitting our model

Here i have used Adam optimizer and mean_squared_error as loss and have trained model for 100 epochs

# Model Training and Results

In [ ]:
#face2sketch
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001), loss = 'mean_absolute_error',
              metrics = ['acc'])

hist = model.fit(train_image, train_sketch_image, epochs = 100, verbose = 1)

plt.plot(hist.history["loss"]);
plt.xlabel('Epochs');
plt.ylabel('Training Error');

In [ ]:
#sketch2face
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001), loss = 'mean_absolute_error',
              metrics = ['acc'])

hist2 = model.fit(train_sketch_image, train_image, epochs = 100, verbose = 1)

plt.plot(hist2.history["loss"]);
plt.xlabel('Epochs');
plt.ylabel('Training Error');

Evaluating our face2sketch model

In [8]:
#face2sketch
prediction_on_test_data = model.evaluate(test_image, test_sketch_image)
print("Loss: ", prediction_on_test_data[0])
print("Accuracy: ", np.round(prediction_on_test_data[1] * 100,1))

4/4 [==============================] - 9s 60ms/step - loss: 0.0771 - acc: 0.3530
Loss:  0.07714570313692093
Accuracy:  35.3


Evaluating our sketch2face model

In [16]:
#sketch2face
prediction_on_test_data = model.evaluate(test_sketch_image, test_image)
print("Loss: ", prediction_on_test_data[0])
print("Accuracy: ", np.round(prediction_on_test_data[1] * 100,1))

4/4 [==============================] - 0s 32ms/step - loss: 0.0784 - acc: 0.8972
Loss:  0.07843179255723953
Accuracy:  89.7


Plotting our predicted sketch along with real sketch

In [9]:
#face2sketch
def show_images(real,sketch, predicted):
    plt.figure(figsize = (12,12))
    plt.subplot(1,3,1)
    plt.title("Image",fontsize = 15, color = 'Lime')
    plt.imshow(real)
    plt.subplot(1,3,2)
    plt.title("sketch",fontsize = 15, color = 'Blue')
    plt.imshow(sketch)
    plt.subplot(1,3,3)
    plt.title("Predicted",fontsize = 15, color = 'gold')
    plt.imshow(predicted)

In [17]:
#sketch2face
def show_images2(sketch, real, predicted):
    plt.figure(figsize = (12,12))
    plt.subplot(1,3,1)
    plt.title("Sketch",fontsize = 15, color = 'Lime')
    plt.imshow(sketch)
    plt.subplot(1,3,2)
    plt.title("Image",fontsize = 15, color = 'Blue')
    plt.imshow(real)
    plt.subplot(1,3,3)
    plt.title("Predicted",fontsize = 15, color = 'gold')
    plt.imshow(predicted)

In [ ]:
#face2sketch
ls = [i for i in range(0,95,8)]
for i in ls:
    predicted =np.clip(model.predict(test_image[i].reshape(1,SIZE,SIZE,3)),0.0,1.0).reshape(SIZE,SIZE,3)
    show_images(test_image[i],test_sketch_image[i],predicted)

In [18]:
#sketch2face
ls = [i for i in range(0,95,8)]
for i in ls:
    predicted =np.clip(model.predict(test_sketch_image[i].reshape(1,SIZE,SIZE,3)),0.0,1.0).reshape(SIZE,SIZE,3)
    show_images2(test_sketch_image[i],test_image[i],predicted)

Output hidden; open in https://colab.research.google.com to view.

In [13]:
#face2sketch
pred = []
for i in range(0,104):
  predicted =np.clip(model.predict(test_image[i].reshape(1,SIZE,SIZE,3)),0.0,1.0).reshape(SIZE,SIZE,3)
  pred.append(predicted)

In [19]:
#sketch2face
pred = []
for i in range(0,104):
  predicted =np.clip(model.predict(test_sketch_image[i].reshape(1,SIZE,SIZE,3)),0.0,1.0).reshape(SIZE,SIZE,3)
  pred.append(predicted)

In [20]:
from skimage.metrics import structural_similarity as ssim1
SSIMarray = []
sumssim = 0
for i in range(0, 104):
  SSIM = ssim1(test_image[i], pred[i], multichannel=True)
  SSIMarray.append(SSIM)
  sumssim+= SSIM

avgssim = sumssim/104
print("SSIM =",avgssim)

from sklearn.metrics import mean_squared_error as mse1
import math
RMSEarray = []
sumrmse = 0

t = np.reshape(test_image, (104,-1))
p = np.reshape(pred, (104,-1))

for i in range(0, 104):
  MSE = mse1(t[i], p[i])
  RMSEarray.append(math.sqrt(MSE))
  sumrmse+= math.sqrt(MSE)

avgrmse = sumrmse/104
print("RMSE =",avgrmse)

from sklearn.metrics import median_absolute_error as mae

maearray = []
summae = 0

for i in range(0, 104):
  MAE = mae(t[i], p[i])
  maearray.append(MAE)
  summae+= MAE

avgrmae = summae/104
print("MAE =",avgrmae)



SSIM = 0.8240065892981835
RMSE = 0.12190834309288986
MAE = 0.05104717565700412


Save the models

In [ ]:
#face2sketch
#model.save('/content/gdrive/MyDrive/MyModels/Conv_AE/face2sketchConvAE.h5')

In [ ]:
#sketch2face
#model.save("/content/gdrive/MyDrive/MyModels/Conv_AE/sketch2faceConvAE.h5")

In [ ]:
#Paizei na kanei prin to architcture
#tf.keras.utils.plot_model(ae_encoder, show_shapes=True)